<a href="https://colab.research.google.com/github/evacs/Project-4-MovieRec/blob/main/Movie_cleaning_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook imports and cleans our data

# Data importation and cleaning


In [ ]:
# !pip install ipython-sql
# !pip install db-sqlite3
# !pip install flask sqlalchemy
# !pip install Flask-SQLAlchemy

In [ ]:
import sqlite3
import pandas as pd
import os
import numpy as np

#addition for file paths
from google.colab import drive

In [ ]:
%load_ext sql
%sql sqlite://

In [ ]:
# Create database in sqlite
conn = sqlite3.connect('movieRatings.db')

In [ ]:
%%sql
CREATE TABLE movies (
    movieId INT PRIMARY KEY,
    title VARCHAR(255),
    genre VARCHAR(255)
);

 * sqlite://
Done.


[]

In [ ]:
# List files in the current directory
files = os.listdir('.')
files

['.config', 'movieRatings.db', 'sample_data']

In [14]:
#all new code

csv_file_path = 'ml-latest-small/movies.csv'
table_name = 'movies'

# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# Copy files from Google Drive
!cp -r /content/gdrive/MyDrive/ml-latest-small/ .

# Read the CSV file using pandas
movies_df = pd.read_csv(csv_file_path)

# Write the data to the SQLite database
movies_df.to_sql(table_name, conn, if_exists='replace', index=False)


Mounted at /content/gdrive


9742

In [15]:
# Verify import
query = f"SELECT * FROM {table_name};"
result = pd.read_sql(query, conn)
print(result)


      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
3                            Comedy|Drama|Romance  
4                  

In [16]:
%%sql
CREATE TABLE links (
    movieId INT,
    imdbId INT,
    tmdbId INT,
    FOREIGN KEY (movieId) REFERENCES movies(movieId)
);


 * sqlite://
Done.


[]

In [17]:
%%sql
CREATE TABLE ratings (
    userId INT,
    movieId INT,
    rating INT,
    timestamp INT,
    FOREIGN KEY (movieId) REFERENCES movies(movieId)
);


 * sqlite://
Done.


[]

In [19]:
%%sql
CREATE TABLE tags (
    userId INT,
    movieId INT,
    tag VARCHAR(255),
    timestamp INT,
    FOREIGN KEY (movieId) REFERENCES movies(movieId)
);


 * sqlite://
(sqlite3.OperationalError) table tags already exists
[SQL: CREATE TABLE tags (
    userId INT,
    movieId INT,
    tag VARCHAR(255),
    timestamp INT,
    FOREIGN KEY (movieId) REFERENCES movies(movieId)
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [20]:
csv_file_path = 'ml-latest-small/ratings.csv'
table_name = 'ratings'

# Read the CSV file using pandas
ratings_df = pd.read_csv(csv_file_path)

# Write the data to the SQLite database
ratings_df.to_sql(table_name, conn, if_exists='replace', index=False)


100836

In [21]:
csv_file_path = 'ml-latest-small/links.csv'
table_name = 'links'

# Read the CSV file using pandas
links_df = pd.read_csv(csv_file_path)

# Write the data to the SQLite database
links_df.to_sql(table_name, conn, if_exists='replace', index=False)


9742

In [22]:
csv_file_path = 'ml-latest-small/tags.csv'
table_name = 'tags'

# Read the CSV file using pandas
tags_df = pd.read_csv(csv_file_path)

# Write the data to the SQLite database
tags_df.to_sql(table_name, conn, if_exists='replace', index=False)


3683

In [23]:
# View a table to varify
table_name = "links"
query = f"SELECT * FROM {table_name};"
result = pd.read_sql(query, conn)
print(result)


      movieId   imdbId    tmdbId
0           1   114709     862.0
1           2   113497    8844.0
2           3   113228   15602.0
3           4   114885   31357.0
4           5   113041   11862.0
...       ...      ...       ...
9737   193581  5476944  432131.0
9738   193583  5914996  445030.0
9739   193585  6397426  479308.0
9740   193587  8391976  483455.0
9741   193609   101726   37891.0

[9742 rows x 3 columns]


In [24]:
# GIVES AN ERROR-IGNORE IT
# Split the genre column into separate columns
genres_split = movies_df['genres'].str.get_dummies('|')

# Concatenate the split genres DataFrame with the original movies DataFrame
movies_df = pd.concat([movies_df, genres_split], axis=1)

# Drop the original genre column
movies_df = movies_df.drop('genres', axis=1)

# Fill NaN values with 0 in the new genre columns
movies_df = movies_df.fillna(0).astype(int)


ValueError: ignored

In [25]:
# View error
movies_df.loc[movies_df['title'] == 'Toy Story (1995)']

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# genres = ["Action", "Adventure","Animation","Children's","Comedy", "Crime", "Documentary", "Drama","Fantasy","Film-Noir", "Horror",
# "Musical","Mystery","Romance","Sci-Fi","Thriller", "War","Western","(no genres listed)"]

In [27]:
# Filter the DataFrame to get movies with 'No genres listed' (1 in the column)
no_genres_movies = movies_df[movies_df['(no genres listed)'] == 1]

# Extract the titles of these movies
titles_with_no_genres = no_genres_movies['title']

# Display the list of titles
movies_df = movies_df[movies_df['(no genres listed)'] != 1]


In [28]:
# Drop IMAX column
movies_df = movies_df.drop('IMAX', axis=1)

# Write to db
movies_df.to_sql('movies', conn, if_exists='replace', index=False)

9708

In [29]:
%%sql
SELECT * FROM links
LIMIT 10;

 * sqlite://
Done.


movieId,imdbId,tmdbId


In [30]:
movies_df = movies_df.drop('(no genres listed)', axis=1)
movies_df.to_sql('movies', conn, if_exists='replace', index=False)

9708

In [31]:


# SQL query to select the first few rows
query = "SELECT * FROM movies LIMIT 5"

# Use Pandas to execute the SQL query and display the result
result = pd.read_sql(query, conn)
result

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
# SQL query to join the "movies" and "ratings" tables based on the "movieId" column
query = """
SELECT m.*, r.userId, r.rating, r.timestamp
FROM movies AS m
INNER JOIN ratings AS r ON m.movieId = r.movieId
"""

# Use Pandas to execute the SQL query and display the result
result = pd.read_sql(query, conn)
result.head()


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,rating,timestamp
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,4.0,964982703
1,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,5,4.0,847434962
2,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,7,4.5,1106635946
3,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,15,2.5,1510577970
4,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,17,4.5,1305696483


In [33]:
# Drop timestamps from all tables
result = result.drop('timestamp', axis=1)
result.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,rating
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,4.0
1,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,5,4.0
2,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,7,4.5
3,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,15,2.5
4,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,17,4.5


In [34]:
# drop tmdb
links_df = links_df.drop('tmdbId', axis=1)

# Write to db
links_df.to_sql('links', conn, if_exists='replace', index=False)

# View table
links_df.head()

,movieId,imdbId
0,1,114709
1,2,113497
2,3,113228
3,4,114885
4,5,113041


In [35]:
# find top 5 movies in each genre

genre_columns = ["Action", "Adventure","Animation","Children","Comedy", "Crime", "Documentary", "Drama","Fantasy","Film-Noir", "Horror",
"Musical","Mystery","Romance","Sci-Fi","Thriller", "War","Western"]

# Create a new DataFrame to store the top 5 movies for each genre
top_movies_df = pd.DataFrame(columns = genre_columns)

# Iterate through each genre
for genre in genre_columns:
    # Select movies of the current genre
    genre_movies = result[result[genre] == 1]

    # Group by movie title and count the number of ratings
    genre_movie_counts = genre_movies.groupby('title').size().reset_index(name='rating_count')

    # Sort by rating count in descending order and select the top 5
    top_genre_movies = genre_movie_counts.sort_values(by='rating_count', ascending=False).head(5)

    # Store data
    top_movies = top_genre_movies['title'].reset_index(drop=True)

    # Populate df with top 5 movies in each category
    # top_movies_list.append(top_movies)
    top_movies_df[genre]=top_movies

top_movies_df


,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,"Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Toy Story (1995),Toy Story (1995),Forrest Gump (1994),"Shawshank Redemption, The (1994)",Bowling for Columbine (2002),Forrest Gump (1994),Toy Story (1995),L.A. Confidential (1997),"Silence of the Lambs, The (1991)",Aladdin (1992),"Usual Suspects, The (1995)",Forrest Gump (1994),"Matrix, The (1999)",Pulp Fiction (1994),Forrest Gump (1994),Dances with Wolves (1990)
1,Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Aladdin (1992),Aladdin (1992),Pulp Fiction (1994),Pulp Fiction (1994),Super Size Me (2004),"Shawshank Redemption, The (1994)","Lord of the Rings: The Fellowship of the Ring,...",Sin City (2005),"Sixth Sense, The (1999)","Lion King, The (1994)",Seven (a.k.a. Se7en) (1995),American Beauty (1999),Star Wars: Episode IV - A New Hope (1977),"Silence of the Lambs, The (1991)",Braveheart (1995),Back to the Future Part III (1990)
2,Jurassic Park (1993),Toy Story (1995),"Lion King, The (1994)","Lion King, The (1994)",Toy Story (1995),"Silence of the Lambs, The (1991)",Fahrenheit 9/11 (2004),Pulp Fiction (1994),"Lord of the Rings: The Two Towers, The (2002)",Chinatown (1974),Alien (1979),Beauty and the Beast (1991),"Sixth Sense, The (1999)",True Lies (1994),Jurassic Park (1993),"Matrix, The (1999)",Schindler's List (1993),Maverick (1994)
3,Braveheart (1995),Star Wars: Episode V - The Empire Strikes Back...,Shrek (2001),Shrek (2001),Aladdin (1992),Fight Club (1999),Roger & Me (1989),Braveheart (1995),"Lord of the Rings: The Return of the King, The...",Mulholland Drive (2001),Aliens (1986),Willy Wonka & the Chocolate Factory (1971),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Speed (1994),Terminator 2: Judgment Day (1991),Jurassic Park (1993),Saving Private Ryan (1998),"Good, the Bad and the Ugly, The (Buono, il bru..."
4,Terminator 2: Judgment Day (1991),Independence Day (a.k.a. ID4) (1996),Beauty and the Beast (1991),Beauty and the Beast (1991),Fargo (1996),"Usual Suspects, The (1995)",Hoop Dreams (1994),Schindler's List (1993),Shrek (2001),Dark City (1998),Interview with the Vampire: The Vampire Chroni...,"Nightmare Before Christmas, The (1993)",Mission: Impossible (1996),Shrek (2001),Star Wars: Episode V - The Empire Strikes Back...,Fight Club (1999),Apocalypse Now (1979),Django Unchained (2012)


In [36]:
top_movies_df.to_sql('top_movies', conn, if_exists='replace', index=False)

5

# Model training

# Flask Stuff

In [42]:
# Copy files from Google Drive
!cp -r /content/gdrive/MyDrive/templates/ .
!cp -r /content/gdrive/MyDrive/static/ .

In [44]:
!pip install flask pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.8/731.8 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.1-py3-none-any.whl size=21122 sha256=f476ef3d6d0374c722b08499f52ba372f45b29026493057d5e14e85f1c34dde9
  Stored in directory: /root/.cache/pip/wheels/3b/32/0e/27789b6fde02bf2b320d6f1a0fd9e1354b257c5f75eefc29bc
Successfully built pyngrok


In [ ]:
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'  # SQLite database in memory
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)

    def __repr__(self):
        return f'<User {self.username}>'

@app.route('/')
def index():
    users = User.query.all()
    return render_template('index.html', users=users)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [55]:
from flask import Flask, jsonify, render_template, request
from sqlalchemy import create_engine, text, func, extract
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

app = Flask(__name__)

# Assuming your SQLAlchemy setup is like this
engine = create_engine("sqlite:///movieRatings.db")
Base = automap_base()
Base.prepare(engine, reflect=True)
# Movies = Base.classes.result  # Assuming the table name is 'movies'
# TopMovies = Base.classes.top_movies



@app.route('/')
def index():
    # movie_categories = ["Action", "Adventure","Animation","Children's","Comedy", "Crime", "Documentary", "Drama","Fantasy","Film-Noir", "Horror",
    #                     "Musical","Mystery","Romance","Sci-Fi","Thriller", "War","Western"]

    return render_template('index.html')#, movie_categories=movie_categories)


# Assuming your SQLAlchemy setup is like this
engine = create_engine("sqlite:///movieRatings.db")
print("hello")

query = text("SELECT * FROM links")

if __name__ == '__main__':
    app.run()

hello
 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-55-a34ab9deaec3>:11: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import flask_ngrok
from flask_ngrok import run_with_ngrok


run_with_ngrok(app)  # Start ngrok when the app is run

# This code block will keep the app running during the Colab session
import threading
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000}).start()
